In [1]:
import requests
import bs4 as bs
import pandas as pd

# Question1
Scrape this url: https://www.rottentomatoes.com

In [0]:
url = "https://www.rottentomatoes.com/top/bestofrt/top_100_animation_movies/"
rtest = requests.get(url)
rtest.encoding = 'utf-8'
soup = bs.BeautifulSoup(rtest.text, 'lxml')

# Question 2.1

In [0]:
title = soup.title.text
print(title)

# Question 2.2

In [0]:
description = soup.find('<meta name = "description">')
print(description)

# Question 2.3

In [0]:
keywords = soup.find('keywords')
print(keywords)

# Question 2.4

In [0]:
h2 = soup.find('h2')
print(h2.text)

# Question 2.5

In [0]:
for p in soup.find_all('p'):
    print(p.text)

# Question3
![Scraping Target](./scraping_target.png)

# Question 4

## to harvest and generate urls autometically

In [0]:
url = "https://www.rottentomatoes.com/top/bestofrt/top_100_animation_movies/"
rtest = requests.get(url)
rtest.encoding = 'utf-8'
soup = bs.BeautifulSoup(rtest.text, 'lxml')

In [0]:
# extract genres in order to find urls automatically
genre = soup.find('ul', attrs = {'class':'dropdown-menu'})
genre_list = [k.text for k in genre.find_all('a')]
for i in range(len(genre_list)):
    genre_list[i] = genre_list[i].strip()
    genre_list[i] = genre_list[i].lower()
    genre_list[i] = genre_list[i].replace(' & ','__')
    genre_list[i] = genre_list[i].replace(' ','_')
    
print(genre_list)

## to harvest ranking, title, year and link in **multiple pages**

In [0]:
title_text = []
title_year = []
ranking_list = []
genre_list_show = []
link_list = []

for genre in genre_list:
    url = "https://www.rottentomatoes.com/top/bestofrt/top_100_"+genre+"_movies/"
    r = requests.get(url)
    r.encoding = 'utf-8'
    soup = bs.BeautifulSoup(r.text, 'lxml')
    
    movie_table_container = soup.find('table', attrs = {'class':'table'})
    #print(movie_table_container)
    
    # title layout: <movie_title><year>
    title = [t.text for t in movie_table_container.find_all(class_ = 'unstyled articleLink')]
    # title is a list of strings
    
    # add 'genre' to genre_list_show
    for i in range(len(title)):
        genre_list_show.append(genre)
        
    # clean 'title_text'
    for i in range(len(title)):
        title_text.append(title[i].replace(title[i][-6:],''))
    for i in range(len(title_text)):
        title_text[i] = title_text[i].strip()
        
    # clean 'year'
    for i in range(len(title)):
        title_year.append(title[i][-6:])
    for i in range(len(title_year)):
        title_year[i] = title_year[i].strip('(')
        title_year[i] = title_year[i].strip(')')

    # get 'link'    
    a_box = movie_table_container.find_all('a')
    href = [h.get('href') for h in a_box]
    for i in range(len(href)):
        href[i] = 'https://www.rottentomatoes.com'+href[i]
        link_list.append(href[i])
    
    ranking = [r.text for r in movie_table_container.find_all(class_ = 'bold')]
    for r in ranking:
        ranking_list.append(int(r.strip('.')))
    #print(genre_list)
    df = pd.DataFrame({'genre':genre_list_show,'ranking':ranking_list,'title':title_text,'year':title_year,'link':link_list})

In [0]:
df

In [0]:
df.to_csv('tomato_best_100_by_genre.csv')